# 所需环境

transformers版本号为2.1.1，pytorch为1.2.0。

In [29]:
import transformers
print(transformers.__version__)

2.1.1


In [30]:
!pwd

/home/weijing/git/GPT2-Chinese/doupo


在requirments.txt中，列出了所需要的其他的包，如下所示。

In [27]:
!cat requirements.txt

cat: requirements.txt: No such file or directory


# 目录结构

当前文件夹中包含的文件如下。其中**doupo**文件夹下包含“斗破苍穹”的示例任务，用以简单说明如何构建字典、tokenization、以及在一个小文件上从头训练一个指定层数的GPT模型；**pretrain**文件夹下包含预训练任务，主要用以说明如何在260万篇新闻文档上预训练一个12层的GPT2；**dataaugmentation**文件夹下包含数据增强任务，主要用以说明如何利用预训练的GPT2模型在较小的文档集上进行微调，并用于数据增强。

In [22]:
%%bash
tree -L 1 .

.
├── config
├── divide
├── format_raw_txt.py
├── generate.sh
├── model
├── outputs
├── outputs_from_scratch
├── rawdata
├── tokenized
├── train.sh
└── vocab.txt

7 directories, 4 files


需要重点加以说明的是，上述列表中的tokenizations文件夹，train_single.py, 和train_on_small_file.py。

`tokenizations`提供了各种tokenization的方法，具体到本代码库中，使用的是该文件夹中的`tokenization_chars.py`文件。

`train_single.py`：当输入的训练数据为一篇完整的长文档（例如小说）或是一个单一的大型文档，此时使用`train_single.py`。有两方面的原因。（1）train_single.py在训练过程中不对training samples进行random shuffle，保持training samples之间的顺序，因此完整的长文档使用train_single.py。（2）对顺序无关的超多的样本，例如包含260万篇新闻的大型训练集，在训练过程中进行random shuffle的代价是巨大的，包括shuffle的代价，重新划分training sample的代价和tokenization的代价，并且在大型训练集上我们进行预训练的轮数有限，1到2轮就能获得一个较好的预训练模型。综合考虑shuffle的代价和收益，因此也将大型训练集中的多个样本拼接为一个单一的大型文档，使用train_single.py。我们在《从头训练一个GPT2模型》一节对此进行更详细的说明。

`train_on_small_file.py`：当输入的训练数据为多个样本，并且这多个样本的总体积较小时，将这些样本汇总到一个文件，每个样本占据一行，然后使用`train_on_small_file.py`。有如下两方面原因：（1）train_on_small_file.py针对每个样本建立一个单独的training sample送入Transformer Encoder中，如果一个样本的长度不足Transformer Encoder的长度（在本代码库中用n_ctx表示），那么不足部分以\[PAD\]来补足；超出n_ctx的部分直接截断丢弃不用。这一点和train_single.py不同，train_single.py中多个较短的样本可能会占据一个Transformer Encoder的输入长度（n_ctx）。（2）train_on_small_file.py会在不同的训练轮次中，对training samples进行random shuffle，以提高训练质量。由于train_on_small_file.py的训练数据体积较小，样本数也较小，因此每轮进行random shuffle是完全可行的。所以，如果是唐诗、宋词、现代诗、意图增强等文本数据，由于其规模较小，完全可以使用`train_on_small_file.py`来完成训练或者微调，这样得到的模型质量要高于`train_single.py`训练得到的模型。我们在《微调GPT2模型进行数据增强》一节对此进行更详细的说明。

In [8]:
!perl -ne 'print "$. $_" if ($.>=18 and $.<=67)' ../train_single.py

18 def build_files(raw_data_path, divide_path, tokenized_data_path, full_tokenizer, num_pieces):
19     if not os.path.exists(tokenized_data_path):
20         os.mkdir(tokenized_data_path)
21     if not os.path.exists(divide_path):
22         os.mkdir(divide_path)
23     print("now time: ", datetime.now())
24     print("begin to divide raw text ...")
25 
26     total_line_num = 0
27     with open(raw_data_path, 'r', encoding='utf8') as f:
28         for line in f:
29             total_line_num+=1
30 
31     writers = [open(divide_path + 'divide_piece_{}.txt'.format(i), 'w') for i in range(0,num_pieces)]
32 
33     with open(raw_data_path, 'r', encoding='utf8') as f:
34         line_num = 0
35         for line in f:
36             writers[line_num % num_pieces].write("%s" % line)
37             line_num += 1
38     
39     for i in range(0, num_pieces):
40         writers[i].close()
41     
42     print('now time: ', datetime.now())
43     print("begin making tokenization ...")
44     f

In [9]:
!ls

config		   model		 tokenized
divide		   outputs		 Train_GPT2_from_scratch.ipynb
format_raw_txt.py  outputs_from_scratch  train.sh
generate.sh	   rawdata		 vocab.txt


In [13]:
!bash train.sh

setting config/vocab.txt and config/model_config.json
I0414 14:48:02.821990 140337405437760 file_utils.py:39] PyTorch version 1.2.0 available.
args:
Namespace(batch_size=32, device='0,1', divide_path='doupo/divide/', epochs=30, fp16=False, fp16_opt_level='O1', gradient_accumulation=1, ignore_intermediate_epoch_model=True, log_step=20, lr=0.00015, max_grad_norm=1.0, model_config='doupo/config/model_config.json', num_pieces=1, output_dir='doupo/model/', pretrained_model='', raw=False, raw_data_path='doupo/rawdata/train.txt', segment=False, stride=512, tokenized_data_path='doupo/tokenized/', tokenizer_path='doupo/config/vocab.txt', warmup_steps=2000)
config:
{
  "attn_pdrop": 0.1,
  "embd_pdrop": 0.1,
  "finetuning_task": null,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "n_ctx": 512,
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 10,
  "n_positions": 512,
  "num_labels": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_he

In [15]:
!bash generate.sh

I0414 19:00:22.363597 140099496916800 file_utils.py:39] PyTorch version 1.2.0 available.
args:
Namespace(batch_size=1, device='0', fast_pattern=False, length=100, model_config='doupo/model/final_model/model_config.json', model_path='doupo/model/final_model/', no_wordpiece=False, nsamples=50, prefix='萧炎', repetition_penalty=1.0, save_samples=True, save_samples_path='doupo/outputs/', segment=False, temperature=0.8, tokenizer_path='doupo/config/vocab.txt', topk=50, topp=0)
I0414 19:00:22.669387 140099496916800 configuration_utils.py:148] loading configuration file doupo/model/final_model/config.json
I0414 19:00:22.669842 140099496916800 configuration_utils.py:168] Model config {
  "attn_pdrop": 0.1,
  "embd_pdrop": 0.1,
  "finetuning_task": null,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "n_ctx": 512,
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 10,
  "n_positions": 512,
  "num_labels": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past":

In [25]:
!ls

config	format_raw_txt.py  model    outputs_from_scratch  tokenized  vocab.txt
divide	generate.sh	   outputs  rawdata		  train.sh
